In [1]:
!pip install pycaret pandas shap


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from pycaret.regression import *
from datetime import timedelta
import numpy as np

# โหลดไฟล์ CSV
file_path = "D:\Term_Project\pm_2.5\cleaned_data (2)-7days.csv"
df = pd.read_csv(file_path)

# แปลง datetime เป็นชนิด datetime
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True, errors="coerce")

# แปลง 'pm_2_5' เป็นตัวเลข
df["pm_2_5"] = pd.to_numeric(df["pm_2_5"], errors="coerce")

# สร้างสำเนา DataFrame เพื่อใช้ทำความสะอาดข้อมูล
df_cleaned = df.copy()

# ฟังก์ชันสำหรับจัดการ outlier และ NaN ด้วย Hybrid Imputation
def hybrid_imputation(series):
    # 1. จัดการกับ outlier
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    median_value = series.median()
    
    # แทนที่ outlier ด้วยค่ามัธยฐาน
    series_clean = series.where((series >= lower_bound) & (series <= upper_bound), median_value)
    
    # 2. เติมค่าที่หายไปด้วยค่ามัธยฐาน
    series_clean = series_clean.fillna(median_value)
    
    # 3. Clean ค่าที่ซ้ำกันโดยไม่ลบออก (เฉพาะกรณีค่าซ้ำติดกัน)
    # กำหนดขนาดของ window สำหรับการคำนวณ rolling median
    window_size = 3
    series_imputed = series_clean.copy()
    
    # ตรวจสอบค่าซ้ำในแบบ consecutive (ติดกัน)
    for i in range(1, len(series_clean)):
        if series_clean.iloc[i] == series_clean.iloc[i - 1]:
            # กำหนดช่วงของ window รอบ ๆ index i
            start = max(0, i - window_size)
            end = min(len(series_clean), i + window_size + 1)
            window_median = series_clean.iloc[start:end].median()
            series_imputed.iloc[i] = window_median
            
    return series_imputed


# ใช้ Hybrid Imputation สำหรับคอลัมน์ 'pm_2_5'
df_cleaned["pm_2_5"] = hybrid_imputation(df_cleaned["pm_2_5"])

# สร้างฟีเจอร์ย้อนหลัง
for lag in [1, 3, 6, 24]:
    df_cleaned[f"pm_2_5_lag_{lag}"] = df_cleaned["pm_2_5"].shift(lag)

# ตั้งค่า PyCaret และแบ่ง train 80% test 20%
exp = setup(df_cleaned, target="pm_2_5", train_size=0.8, session_id=42, normalize=True, feature_selection=True)

# ปรับแต่งโมเดล Random Forest Regressor
model = create_model("rf")

# ---- สร้างข้อมูลสำหรับพยากรณ์ล่วงหน้า 7 วัน (168 ชั่วโมง) ----
last_date = df_cleaned["datetime"].max()
future_dates = [last_date + timedelta(hours=i) for i in range(1, 169)]

# ใช้ค่าเฉลี่ยล่าสุดของความชื้นและอุณหภูมิ
latest_humidity = df_cleaned["humidity"].iloc[-24:].mean()
latest_temperature = df_cleaned["temperature"].iloc[-24:].mean()
latest_pm25 = df_cleaned["pm_2_5"].iloc[-1]

future_data = pd.DataFrame({
    "datetime": future_dates,
    "humidity": [latest_humidity] * len(future_dates),
    "temperature": [latest_temperature] * len(future_dates),
    "pm_2_5_lag_1": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_3": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_6": [latest_pm25] * len(future_dates),
    "pm_2_5_lag_24": [latest_pm25] * len(future_dates)
})

# เพิ่มฟีเจอร์เชิงเวลา
future_data["hour"] = future_data["datetime"].dt.hour
future_data["dayofweek"] = future_data["datetime"].dt.dayofweek

# Recursive forecasting
for i in range(len(future_data)):
    pred = predict_model(model, data=future_data.iloc[i:i+1])
    predicted_value = pred.iloc[0]["prediction_label"]
    if i + 1 < len(future_data):
        future_data.at[i + 1, "pm_2_5_lag_1"] = predicted_value
    if i + 3 < len(future_data):
        future_data.at[i + 3, "pm_2_5_lag_3"] = predicted_value
    if i + 6 < len(future_data):
        future_data.at[i + 6, "pm_2_5_lag_6"] = predicted_value
    if i + 24 < len(future_data):
        future_data.at[i + 24, "pm_2_5_lag_24"] = predicted_value

# ทำนายค่า PM 2.5
predictions = predict_model(model, data=future_data)

# หาชื่อคอลัมน์การทำนายที่ถูกต้อง
prediction_column = [col for col in predictions.columns if "predict" in col.lower()]
if prediction_column:
    prediction_column = prediction_column[0]
    print(predictions[["datetime", prediction_column]])
else:
    print("ไม่พบคอลัมน์การทำนาย ลองแสดงผลทั้งหมด:")
    print(predictions.head())

# บันทึกผลลัพธ์เป็นไฟล์ CSV
predictions.to_csv("D:/Term_Project/pm_2.5/predicted_pm25.csv", index=False)
print("บันทึกผลลัพธ์การพยากรณ์สำเร็จ!")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000747 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1581
[LightGBM] [Info] Number of data points in the train set: 27572, number of used features: 9
[LightGBM] [Info] Start training from score 12.983037


,Description,Value
0,Session id,42
1,Target,pm_2_5
2,Target type,Regression
3,Original data shape,"(34465, 8)"
4,Transformed data shape,"(34465, 2)"
5,Transformed train set shape,"(27572, 2)"
6,Transformed test set shape,"(6893, 2)"
7,Numeric features,6
8,Date features,1
9,Rows with missing values,0.1%


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.9655,21.0337,4.5862,0.7064,0.3263,0.3152
1,3.0225,22.0754,4.6984,0.7134,0.3225,0.3154
2,2.9080,20.5860,4.5372,0.7187,0.3206,0.3095
3,3.0130,22.0572,4.6965,0.6857,0.3242,0.3549
4,2.9530,21.0141,4.5841,0.7187,0.3219,0.3152
5,2.9964,20.4584,4.5231,0.7189,0.3150,0.3017
6,3.0709,21.5694,4.6443,0.7051,0.3266,0.3173
7,2.9632,21.2224,4.6068,0.7008,0.3326,0.3256
8,2.9514,20.6660,4.5460,0.7200,0.3212,0.3703


               datetime  prediction_label
0   2025-01-28 00:00:00         17.528438
1   2025-01-28 01:00:00         17.717172
2   2025-01-28 02:00:00         18.687602
3   2025-01-28 03:00:00         18.808969
4   2025-01-28 04:00:00         18.512258
..                  ...               ...
163 2025-02-03 19:00:00         11.206917
164 2025-02-03 20:00:00         10.974145
165 2025-02-03 21:00:00         10.427998
166 2025-02-03 22:00:00         11.026214
167 2025-02-03 23:00:00         13.342556

[168 rows x 2 columns]
บันทึกผลลัพธ์การพยากรณ์สำเร็จ!
